# For this demo, we will be using Hugging Face as the LLM, ES vector index as the external knowledge base

## Setting up

> Install packages









In [ ]:
pip install elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
pip install transformers torch elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Inserting documents with embeddings

1.   Creating food_consumed list
2.   Creating food_knowledge_base



In [ ]:
import numpy as np
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Initialize ElasticSearch client
es = Elasticsearch(
    "https://my-elasticsearch-project-a610a8.es.us-west-2.aws.elastic.cloud:443",
    api_key="TzRYd09wVUJzNHJ1TnVZQ2VTQWE6dU5TUmhEaGZDX2xTa0NSVjQteWhydw==")

# Initialize Hugging Face model and tokenizer for embedding generation
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Function to generate embeddings using DistilBERT
def generate_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Mean pooling
    return embeddings.numpy()

# Define the mapping for the `foods_consumed` index (with dense vector for embeddings)
def create_consumed_food_index():
    mappings = {
        "mappings": {
            "properties": {
                "food": {"type": "text"},
                "description": {"type": "text"},
                "date_of_consumption": {"type": "date"},
                "embedding": {
                    "type": "dense_vector",
                    "dims": 768  # The dimension of DistilBERT's output
                }
            }
        }
    }
    es.indices.create(index="foods_consumed", body=mappings, ignore=400)  # ignore 400 if index already exists

# Define the mapping for the `food_knowledge_base` index (with dense vector for embeddings)
def create_food_knowledge_base_index():
    mappings = {
        "mappings": {
            "properties": {
                "food": {"type": "text"},
                "description": {"type": "text"},
                "embedding": {
                    "type": "dense_vector",
                    "dims": 768  # The dimension of DistilBERT's output
                }
            }
        }
    }
    es.indices.create(index="food_knowledge_base", body=mappings, ignore=400)  # ignore 400 if index already exists

# Create indexes (run only once)
create_consumed_food_index()
create_food_knowledge_base_index()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

<ipython-input-7-bb5d29fb185c>:40: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="foods_consumed", body=mappings, ignore=400)  # ignore 400 if index already exists
<ipython-input-7-bb5d29fb185c>:56: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="food_knowledge_base", body=mappings, ignore=400)  # ignore 400 if index already exists


In [ ]:
# Example data for the food knowledge base
food_knowledge_base =[
    {"food": "Pizza", "description": "Classic margherita pizza with extra cheese and basil"},
    {"food": "Sushi", "description": "Fresh salmon and tuna sushi rolls with wasabi and soy sauce"},
    {"food": "Burger", "description": "Juicy beef patty with lettuce, tomato, and cheddar cheese on a sesame bun"},
    {"food": "Pasta", "description": "Spaghetti served with marinara sauce and garlic bread"},
    {"food": "Tacos", "description": "Soft corn tortillas filled with grilled chicken, salsa, and guacamole"},
    {"food": "Lasagna", "description": "Layers of pasta, ricotta cheese, ground beef, and marinara sauce"},
    {"food": "Salad", "description": "Mixed greens with cherry tomatoes, cucumbers, and a balsamic vinaigrette"},
    {"food": "Steak", "description": "Grilled ribeye steak served with mashed potatoes and steamed broccoli"},
    {"food": "Fried Chicken", "description": "Crispy fried chicken served with coleslaw and cornbread"},
    {"food": "Pancakes", "description": "Fluffy pancakes topped with maple syrup and a dusting of powdered sugar"},
    {"food": "Ramen", "description": "Japanese noodle soup with pork, eggs, and green onions in a savory broth"},
    {"food": "Pad Thai", "description": "Stir-fried rice noodles with shrimp, tofu, peanuts, and a tangy sauce"},
    {"food": "Shawarma", "description": "Middle Eastern dish of spiced lamb or chicken wrapped in pita with garlic sauce"},
    {"food": "Ice Cream", "description": "Rich and creamy vanilla ice cream with a swirl of chocolate syrup"},
    {"food": "Cheesecake", "description": "Creamy cheesecake with a graham cracker crust and fruit topping"},
    {"food": "Fajitas", "description": "Sizzling grilled beef or chicken served with bell peppers and onions, with tortillas"},
    {"food": "Fish and Chips", "description": "Battered fish fillets served with crispy fries and tartar sauce"},
    {"food": "Vegetable Stir-Fry", "description": "Assorted vegetables stir-fried in a soy sauce and garlic mix, served with rice"},
    {"food": "Burrito", "description": "Flour tortilla filled with rice, beans, beef or chicken, and sour cream"},
    {"food": "Spring Rolls", "description": "Crispy rolls stuffed with vegetables and served with dipping sauce"},
    {"food": "Waffles", "description": "Golden waffles served with butter, syrup, and fresh berries"},
    {"food": "Curry", "description": "Spicy curry dish with chicken, vegetables, and rice"},
    {"food": "Gyros", "description": "Greek pita sandwich with spiced lamb, tomato, onions, and tzatziki sauce"},
    {"food": "Mushroom Risotto", "description": "Creamy risotto with sautéed mushrooms and Parmesan cheese"},
    {"food": "Pho", "description": "Vietnamese noodle soup with beef, herbs, and a flavorful broth"},
    {"food": "Chicken Caesar Salad", "description": "Grilled chicken breast on a bed of romaine lettuce with Caesar dressing"},
    {"food": "Clams Chowder", "description": "Creamy soup made with clams, potatoes, onions, and celery"},
    {"food": "Baked Ziti", "description": "Pasta baked with marinara sauce and melted mozzarella cheese"},
    {"food": "Tom Yum Soup", "description": "Spicy Thai soup with shrimp, mushrooms, and aromatic herbs"},
    {"food": "Mac and Cheese", "description": "Cheddar cheese sauce over macaroni pasta, baked to perfection"},
    {"food": "Falafel", "description": "Crispy chickpea patties served with pita and a yogurt tahini sauce"},
    {"food": "Beef Tacos", "description": "Ground beef in soft corn tortillas with cheese, lettuce, and salsa"},
    {"food": "Quiche", "description": "Savory tart filled with eggs, cream, cheese, and vegetables"},
    {"food": "Pork Schnitzel", "description": "Breaded and fried pork cutlet, served with lemon wedges and potatoes"},
    {"food": "Chicken Alfredo", "description": "Pasta with creamy Alfredo sauce and grilled chicken breast"},
    {"food": "Croissant", "description": "Flaky, buttery pastry, perfect for breakfast or as a snack"},
    {"food": "Beef Wellington", "description": "Beef tenderloin coated with mushroom duxelles and wrapped in puff pastry"},
    {"food": "Grilled Cheese Sandwich", "description": "Melted cheese between two slices of buttered, toasted bread"},
    {"food": "Caesar Wrap", "description": "Grilled chicken, romaine lettuce, and Caesar dressing wrapped in a flour tortilla"},
    {"food": "Tuna Salad", "description": "Tuna mixed with mayonnaise, onions, and celery, served on a bed of greens"},
    {"food": "Peking Duck", "description": "Crispy-skinned duck served with pancakes, hoisin sauce, and scallions"},
    {"food": "Chicken Kiev", "description": "Chicken breast filled with garlic butter, breaded and fried to perfection"},
    {"food": "Bangers and Mash", "description": "Traditional British dish of sausages served with mashed potatoes and gravy"},
    {"food": "Sauerbraten", "description": "German pot roast marinated in vinegar and spices, served with potatoes and red cabbage"},
    {"food": "Baked Potatoes", "description": "Hot, fluffy potatoes topped with sour cream, chives, and bacon bits"},
    {"food": "Lobster Roll", "description": "Fresh lobster meat in a buttery toasted roll with mayo and lemon"},
    {"food": "Pavlova", "description": "Meringue-based dessert topped with whipped cream and fresh fruit"},
    {"food": "Tiramisu", "description": "Italian dessert made of layers of coffee-soaked ladyfingers and mascarpone cream"},
    {"food": "Churros", "description": "Fried dough pastries rolled in cinnamon sugar and served with chocolate dipping sauce"},
    {"food": "Moussaka", "description": "Layered Greek casserole made with eggplant, ground beef, and béchamel sauce"},
    {"food": "Empanadas", "description": "Flaky pastry pockets filled with spiced beef, chicken, or vegetables"},
    {"food": "Ceviche", "description": "Fresh fish marinated in citrus juice and mixed with onions, cilantro, and chili"},
    {"food": "Sliders", "description": "Miniature burgers served on soft buns with pickles and cheese"},
    {"food": "Peking Duck Pancakes", "description": "Crispy duck served in soft pancakes with cucumber, scallions, and hoisin sauce"},
    {"food": "Crab Cakes", "description": "Golden-brown crab meat patties served with remoulade sauce"},
    {"food": "Risotto", "description": "Creamy rice dish cooked with broth, Parmesan, and your choice of vegetables or meat"},
    {"food": "Baklava", "description": "Sweet pastry made of layers of filo dough, filled with nuts and honey syrup"},
    {"food": "Chicken Tenders", "description": "Breaded and fried chicken strips, served with dipping sauce"},
    {"food": "Tomato Soup", "description": "Rich, creamy soup made with fresh tomatoes and herbs, served with grilled cheese"},
    {"food": "Biryani", "description": "Fragrant rice dish made with spiced meat, vegetables, and herbs"},
    {"food": "Samosas", "description": "Fried pastry filled with spiced potatoes and peas, served with mint chutney"},
    {"food": "Pupusas", "description": "Salvadoran stuffed corn tortillas filled with cheese, beans, and pork"},
    {"food": "Cottage Pie", "description": "Ground beef and vegetables topped with mashed potatoes and baked to golden perfection"}
]
# Generate embeddings for the knowledge base food descriptions
knowledge_base_descriptions = [food["description"] for food in food_knowledge_base]
knowledge_base_embeddings = generate_embeddings(knowledge_base_descriptions)

# Ingest the knowledge base data into ElasticSearch (index: 'food_knowledge_base')
def generate_bulk_data_for_knowledge_base(food_data, embeddings):
    for i, (food, embedding) in enumerate(zip(food_data, embeddings)):
        yield {
            "_index": "food_knowledge_base",
            "_id": i,
            "_source": {
                "food": food["food"],
                "description": food["description"],
                "embedding": embedding.tolist()  # Convert numpy array to list for JSON serialization
            }
        }

# Ingest food knowledge base into ElasticSearch
bulk(es, generate_bulk_data_for_knowledge_base(food_knowledge_base, knowledge_base_embeddings))

(63, [])

In [ ]:
# Function to ingest food consumption history (index: 'foods_consumed')
def generate_bulk_data_for_consumed_food(foods, embeddings):
    for i, (food, embedding) in enumerate(zip(foods, embeddings)):
        yield {
            "_index": "foods_consumed",
            "_id": i,
            "_source": {
                "food": food["food"],
                "description": food["description"],
                "date_of_consumption": food["date_of_consumption"],
                "embedding": embedding.tolist()
            }
        }

# Example food consumption data
food_consumption = [
    {"food": "Pizza", "description": "Classic margherita pizza with extra cheese", "date_of_consumption": "2025-02-21"},
    {"food": "Burger", "description": "Juicy beef patty with lettuce, tomato, and cheese", "date_of_consumption": "2025-02-22"},
    {"food": "Pasta", "description": "Spaghetti with marinara sauce and garlic bread", "date_of_consumption": "2025-02-23"},
    {"food": "Tacos", "description": "Soft tacos with grilled chicken, salsa, and guacamole", "date_of_consumption": "2025-02-24"},
    # Add more consumed food data...
]

# Generate embeddings for the consumed food descriptions
consumed_food_descriptions = [food["description"] for food in food_consumption]
consumed_food_embeddings = generate_embeddings(consumed_food_descriptions)

# Ingest consumed food data into ElasticSearch
bulk(es, generate_bulk_data_for_consumed_food(food_consumption, consumed_food_embeddings))

# Retrieve food consumed in past 5 days and sugggest food that user will like (based on similarity score), comparing preference vector with knowledge base

In [ ]:
# Function to retrieve food consumed in the past 5 days
def get_food_data_last_5_days():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5)

    start_date_str = start_date.isoformat()
    end_date_str = end_date.isoformat()

    query = {
        "query": {
            "range": {
                "date_of_consumption": {
                    "gte": start_date_str,
                    "lte": end_date_str
                }
            }
        },
        "_source": ["food", "description", "embedding"]
    }

    response = es.search(index="foods_consumed", body=query)

    foods_in_last_5_days = []
    for hit in response["hits"]["hits"]:
        food_data = hit["_source"]
        foods_in_last_5_days.append({
            "food": food_data["food"],
            "description": food_data["description"],
            "embedding": np.array(food_data["embedding"])
        })

    return foods_in_last_5_days

# Retrieve food data consumed in the past 5 days
foods_in_last_5_days = get_food_data_last_5_days()

# Compute the preference vector based on the foods you've consumed
def compute_preference_vector(foods):
    embeddings = [food["embedding"] for food in foods]
    preference_vector = np.mean(embeddings, axis=0)
    preference_vector = preference_vector / np.linalg.norm(preference_vector)  # Normalize the vector
    return preference_vector

# Compute preference vector
preference_vector = compute_preference_vector(foods_in_last_5_days)

# Function to retrieve food data from the knowledge base (index: 'food_knowledge_base')
def get_food_data_from_knowledge_base():
    query = {
        "size": 1000,  # Adjust this based on your knowledge base size
        "_source": ["food", "description", "embedding"]
    }

    response = es.search(index="food_knowledge_base", body=query)

    food_data = []
    for hit in response["hits"]["hits"]:
        food_data.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"],
            "embedding": np.array(hit["_source"]["embedding"])
        })

    return food_data

# Function to recommend food based on the preference vector
def recommend_foods(preference_vector, top_n=5):
    all_food_data = get_food_data_from_knowledge_base()
    similarities = []

    for food in all_food_data:
        similarity = cosine_similarity([preference_vector], [food["embedding"]])[0][0]
        similarities.append({
            "food": food["food"],
            "description": food["description"],
            "similarity": similarity
        })

    sorted_recommendations = sorted(similarities, key=lambda x: x["similarity"], reverse=True)
    return sorted_recommendations[:top_n]

# Get top 5 recommended foods
recommended_foods = recommend_foods(preference_vector, top_n=3)

# Display recommended foods
for food in recommended_foods:
    print(f"Food: {food['food']}, Description: {food['description']}, Similarity: {food['similarity']}")

Food: Lasagna, Description: Layers of pasta, ricotta cheese, ground beef, and marinara sauce, Similarity: 0.9661649211309982
Food: Baked Ziti, Description: Pasta baked with marinara sauce and melted mozzarella cheese, Similarity: 0.9576309324309851
Food: Beef Tacos, Description: Ground beef in soft corn tortillas with cheese, lettuce, and salsa, Similarity: 0.9567067286625031


# Getting direct output from query script without using LLM as the output layer

1.   Getting user input and output similar food from knowledge base
2.   Reccomending food based on user's consumed food from past 3 days, and not to suggest anything that user ate in the past 3 days



In [ ]:
# Function to retrieve food data from the knowledge base (index: 'food_knowledge_base')
def get_food_data_from_knowledge_base():
    query = {
        "size": 1000,  # Adjust this based on your knowledge base size
        "_source": ["food", "description", "embedding"]
    }

    response = es.search(index="food_knowledge_base", body=query)

    food_data = []
    for hit in response["hits"]["hits"]:
        food_data.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"],
            "embedding": np.array(hit["_source"]["embedding"])
        })

    return food_data

# Function to process the user's query and return food recommendations
def recommend_food_based_on_query(user_query, top_n=5):
    # Generate embedding for the user query
    query_embedding = generate_embeddings([user_query])[0]

    # Retrieve all food data from the knowledge base
    all_food_data = get_food_data_from_knowledge_base()

    # Calculate the cosine similarity between the user query and each food in the knowledge base
    similarities = []
    for food in all_food_data:
        similarity = cosine_similarity([query_embedding], [food["embedding"]])[0][0]
        similarities.append({
            "food": food["food"],
            "description": food["description"],
            "similarity": similarity
        })

    # Sort foods by similarity score in descending order
    sorted_recommendations = sorted(similarities, key=lambda x: x["similarity"], reverse=True)

    # Return the top_n recommended foods
    return sorted_recommendations[:top_n]

# Function to interact with the user
def ask_user_for_food_query():
    print("What type of food would you like to know about from the knowledge base?")
    user_query = input("Ask your question: ")

    recommended_foods = recommend_food_based_on_query(user_query)

    # Display the recommendations
    print("\nHere are the top food recommendations based on your query:")
    for i, food in enumerate(recommended_foods, 1):
        print(f"{i}. Food: {food['food']}, Description: {food['description']}, Similarity: {food['similarity']:.4f}")

# Run the function to ask the user for a food-related query
ask_user_for_food_query()

What type of food would you like to know about from the knowledge base?
Ask your question: noodle dishes

Here are the top food recommendations based on your query:
1. Food: Pho, Description: Vietnamese noodle soup with beef, herbs, and a flavorful broth, Similarity: 0.7958
2. Food: Ramen, Description: Japanese noodle soup with pork, eggs, and green onions in a savory broth, Similarity: 0.7886
3. Food: Pad Thai, Description: Stir-fried rice noodles with shrimp, tofu, peanuts, and a tangy sauce, Similarity: 0.7716
4. Food: Risotto, Description: Creamy rice dish cooked with broth, Parmesan, and your choice of vegetables or meat, Similarity: 0.7678
5. Food: Burger, Description: Juicy beef patty with lettuce, tomato, and cheddar cheese on a sesame bun, Similarity: 0.7662


In [ ]:
# Function to retrieve consumed food data from the foods_consumed index
def get_consumed_foods_from_elastic():
    # Get the current date
    today = datetime.now()

    # Calculate the date 3 days ago
    three_days_ago = today - timedelta(days=3)

    # Convert to string in the format Elasticsearch expects (yyyy-mm-dd)
    three_days_ago_str = three_days_ago.strftime('%Y-%m-%d')

    # Elasticsearch query to fetch food consumed in the last 3 days
    query = {
        "query": {
            "range": {
                "date_of_consumption": {
                    "gte": three_days_ago_str
                }
            }
        },
        "_source": ["food", "description", "date_of_consumption"]
    }

    # Execute the search query on the 'foods_consumed' index
    response = es.search(index="foods_consumed", body=query)

    consumed_foods = []
    for hit in response["hits"]["hits"]:
        consumed_foods.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"]
        })

    return consumed_foods

# Function to retrieve food data from the knowledge base (index: 'food_knowledge_base')
def get_food_data_from_knowledge_base():
    query = {
        "size": 1000,  # Adjust this based on your knowledge base size
        "_source": ["food", "description", "embedding"]
    }

    response = es.search(index="food_knowledge_base", body=query)

    food_data = []
    for hit in response["hits"]["hits"]:
        food_data.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"],
            "embedding": np.array(hit["_source"]["embedding"])
        })

    return food_data

# Function to recommend food based on your preferences and avoid repeats
def recommend_food_based_on_preferences(consumed_foods, top_n=5):
    # Generate embeddings for the foods you've consumed
    consumed_food_descriptions = [food["description"] for food in consumed_foods]
    consumed_embeddings = generate_embeddings(consumed_food_descriptions)

    # Compute your preference vector by averaging the embeddings of consumed foods
    preference_vector = np.mean(consumed_embeddings, axis=0)

    # Retrieve all food data from the knowledge base
    all_food_data = get_food_data_from_knowledge_base()

    # Calculate the cosine similarity between the preference vector and each food in the knowledge base
    similarities = []
    for food in all_food_data:
        similarity = cosine_similarity([preference_vector], [food["embedding"]])[0][0]
        similarities.append({
            "food": food["food"],
            "description": food["description"],
            "similarity": similarity
        })

    # Sort foods by similarity score in descending order
    sorted_recommendations = sorted(similarities, key=lambda x: x["similarity"], reverse=True)

    # Filter out the foods you've already consumed in the last 3 days
    consumed_food_names = [food["food"] for food in consumed_foods]
    filtered_recommendations = [rec for rec in sorted_recommendations if rec["food"] not in consumed_food_names]

    # Return the top_n recommended foods
    return filtered_recommendations[:top_n]

# Function to interact with the user and display recommendations
def recommend_based_on_past_3_days():
    # Get the foods you consumed in the last 3 days
    consumed_foods = get_consumed_foods_from_elastic()

    # Check if we have enough food data
    if len(consumed_foods) == 0:
        print("You haven't consumed any food in the last 3 days.")
        return

    # Get food recommendations based on your preferences and consumed food data
    recommended_foods = recommend_food_based_on_preferences(consumed_foods)

    # Display the recommendations
    print("Here are the top food recommendations based on your preferences (not repeated from past 3 days):")
    for i, food in enumerate(recommended_foods, 1):
        print(f"{i}. Food: {food['food']}, Description: {food['description']}, Similarity: {food['similarity']:.4f}")

# Run the function to get food recommendations based on past 3 days
recommend_based_on_past_3_days()

Here are the top food recommendations based on your preferences (not repeated from past 3 days):
1. Food: Beef Tacos, Description: Ground beef in soft corn tortillas with cheese, lettuce, and salsa, Similarity: 0.9695
2. Food: Lasagna, Description: Layers of pasta, ricotta cheese, ground beef, and marinara sauce, Similarity: 0.9624
3. Food: Caesar Wrap, Description: Grilled chicken, romaine lettuce, and Caesar dressing wrapped in a flour tortilla, Similarity: 0.9578
4. Food: Fajitas, Description: Sizzling grilled beef or chicken served with bell peppers and onions, with tortillas, Similarity: 0.9544
5. Food: Moussaka, Description: Layered Greek casserole made with eggplant, ground beef, and béchamel sauce, Similarity: 0.9533


# Feeding the embedded result into LLM and using LLM to provide the response

In [ ]:
from datetime import datetime, timedelta
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertModel, GPT2LMHeadModel, GPT2Tokenizer
from elasticsearch import Elasticsearch
import torch


# Initialize Hugging Face tokenizer and model for embedding generation (DistilBERT for embeddings)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Initialize GPT-2 model and tokenizer for generating responses
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set pad_token to eos_token for GPT-2
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Function to generate embeddings using Hugging Face DistilBert
def generate_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Function to get the foods consumed in the last 3 days
def get_consumed_foods_from_elastic():
    today = datetime.now()
    three_days_ago = today - timedelta(days=3)
    three_days_ago_str = three_days_ago.strftime('%Y-%m-%d')

    query = {
        "query": {
            "range": {
                "date_of_consumption": {
                    "gte": three_days_ago_str
                }
            }
        },
        "_source": ["food", "description", "date_of_consumption"]
    }

    response = es.search(index="foods_consumed", body=query)

    consumed_foods = []
    for hit in response["hits"]["hits"]:
        consumed_foods.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"]
        })

    return consumed_foods

# Function to retrieve food data from the knowledge base (index: 'food_knowledge_base')
def get_food_data_from_knowledge_base():
    query = {
        "size": 1000,
        "_source": ["food", "description", "embedding"]
    }

    response = es.search(index="food_knowledge_base", body=query)

    food_data = []
    for hit in response["hits"]["hits"]:
        food_data.append({
            "food": hit["_source"]["food"],
            "description": hit["_source"]["description"],
            "embedding": np.array(hit["_source"]["embedding"])
        })

    return food_data

# Function to recommend food based on your preferences and avoid repeats
def recommend_food_based_on_preferences(consumed_foods, top_n=5):
    consumed_food_descriptions = [food["description"] for food in consumed_foods]
    consumed_embeddings = generate_embeddings(consumed_food_descriptions)

    preference_vector = np.mean(consumed_embeddings, axis=0)

    all_food_data = get_food_data_from_knowledge_base()

    similarities = []
    for food in all_food_data:
        similarity = cosine_similarity([preference_vector], [food["embedding"]])[0][0]
        similarities.append({
            "food": food["food"],
            "description": food["description"],
            "similarity": similarity
        })

    sorted_recommendations = sorted(similarities, key=lambda x: x["similarity"], reverse=True)

    consumed_food_names = [food["food"] for food in consumed_foods]
    filtered_recommendations = [rec for rec in sorted_recommendations if rec["food"] not in consumed_food_names]

    return filtered_recommendations[:top_n]

# Function to get a friendly response from GPT-2 based on recommendations
def get_llm_response_from_gpt2(recommendations):
    recommendations_str = "\n".join(
        [f"{i+1}. {rec['food']}: {rec['description']} (Similarity: {rec['similarity']:.4f})" for i, rec in enumerate(recommendations)]
    )

    prompt = f"""
    Based on the foods the user has eaten recently, here are some food recommendations:
    {recommendations_str}

    Please summarize these recommendations in a friendly, conversational tone.
    Suggest one or more food that the user might enjoy trying next, ensuring the suggestions come from the list above.

    """

    # Tokenize the input prompt
    inputs = gpt2_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    # Add attention_mask and pad_token_id explicitly
    inputs['attention_mask'] = torch.ones(inputs['input_ids'].shape, dtype=torch.long)  # Create attention mask (1 for all tokens)
    inputs['pad_token_id'] = gpt2_tokenizer.eos_token_id  # Set pad_token_id to eos_token_id

    # Generate the model's response
    outputs = gpt2_model.generate(
        inputs['input_ids'],
        max_length=500,
        num_return_sequences=1,
        no_repeat_ngram_size=2,

        attention_mask=inputs['attention_mask'],  # Pass attention mask explicitly
        pad_token_id=gpt2_tokenizer.eos_token_id  # Use eos_token_id as pad_token_id
    )

    # Decode the output (we want to skip the prompt part)
    response = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Do not remove the recommendations, let the response include the entire model's output
    return response

# Function to interact with the user and display recommendations
def recommend_based_on_past_3_days():
    consumed_foods = get_consumed_foods_from_elastic()

    if len(consumed_foods) == 0:
        print("You haven't consumed any food in the last 3 days.")
        return

    recommended_foods = recommend_food_based_on_preferences(consumed_foods)

    # Get LLM response from GPT-2
    llm_response = get_llm_response_from_gpt2(recommended_foods)

    # Display the LLM's response
    print("LLM's friendly response to the user:")
    print(llm_response)

# Run the function to get food recommendations and get LLM reply
recommend_based_on_past_3_days()



LLM's friendly response to the user:

    Based on the foods the user has eaten recently, here are some food recommendations:
    1. Beef Tacos: Ground beef in soft corn tortillas with cheese, lettuce, and salsa (Similarity: 0.9695)
2. Lasagna: Layers of pasta, ricotta cheese, ground beef, and marinara sauce (Similarity: 0.9624)
3. Caesar Wrap: Grilled chicken, romaine lettuce, and Caesar dressing wrapped in a flour tortilla (Similarity: 0.9578)
4. Fajitas: Sizzling grilled beef or chicken served with bell peppers and onions, with tortillas (Similarity: 0.9544)
5. Moussaka: Layered Greek casserole made with eggplant, ground beef, and béchamel sauce (Similarity: 0.9533)
    
    Please summarize these recommendations in a friendly, conversational tone.
    Suggest one or more food that the user might enjoy trying next, ensuring the suggestions come from the list above. 
    
     I'm not sure if this is a good idea, but I think it's a great idea. I've tried it, I love it. It's so good. 